In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import torch
from omegaconf import OmegaConf
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

from text_recognizer.data.iam_lines import IAMLines

In [3]:
!ulimit -n 65000

zsh:ulimit:1: value exceeds hard limit


In [4]:
def load_config(path: Path):
    with initialize(config_path=path.parent):
        cfg = compose(config_name=path.name)
    return cfg

In [5]:
datamodule_path = Path("../training/conf/datamodule/iam_lines.yaml")
mapping_path = Path("../training/conf/mapping/word_piece.yaml")
mapping_path = Path("../training/conf/mapping/characters.yaml")
data_cfg = load_config(datamodule_path)
mapping_cfg = load_config(mapping_path)

In [7]:
data_cfg.batch_size = 8
data_cfg.num_workers = 1

In [8]:
datamodule = instantiate(data_cfg, mapping=mapping_cfg)
datamodule.prepare_data()
datamodule.setup()
print(datamodule)

2021-10-11 22:00:17.870 | DEBUG    | text_recognizer.data.transforms.load_transform:_load_config:17 - Loading transforms from config: transform/iam_lines.yaml
2021-10-11 22:00:20.796 | DEBUG    | text_recognizer.data.transforms.load_transform:_load_config:17 - Loading transforms from config: test_transform/iam_lines.yaml


IAM Lines dataset
Num classes: 58
Input dims: (1, 56, 1024)
Output dims: (89, 1)
Train/val/test sizes: 9116, 2279, 1958
Train Batch x stats: (torch.Size([8, 1, 56, 1024]), torch.float32, tensor(0.), tensor(0.0269), tensor(0.0916), tensor(1.))
Train Batch y stats: (torch.Size([8, 89]), torch.int64, tensor(1), tensor(54))
Test Batch x stats: (torch.Size([8, 1, 56, 1024]), torch.float32, tensor(0.), tensor(0.0327), tensor(0.0940), tensor(0.8588))
Test Batch y stats: (torch.Size([8, 89]), torch.int64, tensor(1), tensor(52))



In [ ]:
def convert_y_label_to_string(y, datamodule=datamodule):
    return ''.join([datamodule.mapping[i] for i in y if i != 3])

convert_y_label_to_string(dataset[0][1])

In [ ]:
X, Y = next(iter(datamodule.train_dataloader()))
for i in range(8):
    plt.figure(figsize=(40, 20))
    x, y = X[i], Y[i]
    sentence = convert_y_label_to_string(y)
    plt.imshow(x.squeeze(), cmap='gray', vmin=0, vmax=1)
    plt.title(sentence)